In [ ]:
import os

from langchain.agents import AgentType, initialize_agent
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper
from langchain_openai import ChatOpenAI

In [ ]:
# Set your environment variables using os.environ
os.environ["GITHUB_APP_ID"]
os.environ["GITHUB_APP_PRIVATE_KEY"] 
os.environ["GITHUB_REPOSITORY"] 
os.environ["GITHUB_BRANCH"] 
os.environ["GITHUB_BASE_BRANCH"] 
os.environ["OPENAI_API_KEY"] 



In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")
github = GitHubAPIWrapper()
toolkit = GitHubToolkit.from_github_api_wrapper(github)
tools = toolkit.get_tools()

# STRUCTURED_CHAT includes args_schema for each tool, helps tool args parsing errors.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
print("Available tools:")
for tool in tools:
    print("\t" + tool.name)

In [ ]:
agent.run(
    "You have the software engineering capabilities of a Google Principle engineer. You are tasked with completing issues on a github repository. Please look at the existing issues and complete them."
)

In [ ]:
gh_issue_prompt_template = """Please implement these changes by creating or editing the necessary files. 

1. First use read_file to read any files in the repo that seem relevant. 
2. Then, when you're ready, start implementing changes by creating and updating files. Implement any and all remaining code to make the project work as the commenter intended. 
2. The last step is to create a PR with a clear and concise title and description, list any concerns or final changes necessary in the PR body.
3. After opening the PR, comment on the original issue and mention the new PR your just opened, you must comment "I opened a PR for you to review here #<PR_NUMBER>" (it'll be something like #30). That hashtag syntax will automatically link to the PR, as necessary. Thanks.
4. If you feel the PR is satisfactory for completing your assignment, create a review request for the original user that opened the issue. Use their username to tag them.

Feel free to ask for help or leave a comment on the Issue or PR if you're stuck.

Here's your latest assignment: {issue_description}"""

print(gh_issue_prompt_template)

In [ ]:
def format_issue(issue):
    title = f"Title: {issue.get('title')}."
    opened_by = f"Opened by user: {issue.get('opened_by')}"
    body = f"Body: {issue.get('body')}"
    comments = issue.get("comments")  # often too long
    return "\n".join([title, opened_by, body])


issue = github.get_issue(27)  # task to implement a RNA-seq pipeline (bioinformatics)
final_gh_issue_prompt = gh_issue_prompt_template.format(
    issue_description=format_issue(issue)
)
print(final_gh_issue_prompt)

In [ ]:
from langchain.memory.summary_buffer import ConversationSummaryBufferMemory
from langchain_core.prompts.chat import MessagesPlaceholder

summarizer_llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")  # type: ignore
chat_history = MessagesPlaceholder(variable_name="chat_history")
memory = ConversationSummaryBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    llm=summarizer_llm,
    max_token_limit=16_000,
)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,  # or pass a function that accepts the error and returns a string
    max_iterations=30,
    max_execution_time=None,
    early_stopping_method="generate",
    memory=memory,
    # trim_intermediate_steps=fancier_trim_intermediate_steps,
    agent_kwargs={
        "memory_prompts": [chat_history],
        "input_variables": ["input", "agent_scratchpad", "chat_history"],
        "prefix": final_gh_issue_prompt,
    },
)

In [ ]:
agent.run(final_gh_issue_prompt)